# 使用前，需要先导入需要的头文件

In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [2]:
class MyDataset : public torch::data::Dataset<MyDataset>
{
    private:
        torch::Tensor states_, labels_;

    public:
        explicit MyDataset(torch::Tensor states, torch::Tensor labels) 
            : states_(states),
              labels_(labels) {   };

        torch::data::Example<> get(size_t index) override {
            return {states_[index], labels_[index]};
        };

        torch::optional<size_t> size() const override {
            return states_.size(0);
        };
};

# 模型选择、欠拟合和过拟合

### 使用以下三阶多项式来生成训练和测试数据的标签

$y = 5 + 1.2x - 3.4 \frac{x^2}{2!} + 5.6 \frac{x^3}{3!} + \epsilon \quad where \quad\epsilon \sim \mathcal{N} (0,0.01^2)$

In [3]:
constexpr int max_degree = 20;
constexpr int n_train = 100;
constexpr int n_test = 100;

//多项式系数
torch::Tensor true_w = torch::zeros(max_degree);
float temp[] = {5.0, 1.2, -3.4, 5.6};
memcpy(true_w.data_ptr(), temp, sizeof(temp));

//准备输入
torch::Tensor features = torch::randint(-99999, 99999, {n_train + n_test, 1}) / 100000.0;
torch::Tensor poly_features  = features.clone();
poly_features = poly_features.pow(torch::arange(max_degree));

for (int i = 0; i < max_degree; i++) {
    int factorial = 1;
    if(i != 0) factorial = tgamma(i);
    for (int row = 0; row < (n_train + n_test); row++) {
        poly_features[row][i] /= factorial;
    }
}

// printT(features);
// printT(poly_features);

//生成输出
true_w = true_w.reshape({max_degree, 1});
// printT(true_w);
torch::Tensor labels = poly_features.mm(true_w);
labels += torch::rand_like(labels) * 0.1;

In [4]:
printT(features.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}));    
  
printT(poly_features.index({torch::indexing::Slice(torch::indexing::None, 1, torch::indexing::None)}));    

printT(labels.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}));    

features.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}) = 
 0.6649
 0.0729
[ CPUFloatType{2,1} ]
<<--->>

poly_features.index({torch::indexing::Slice(torch::indexing::None, 1, torch::indexing::None)}) = 
Columns 1 to 6 1.0000e+00  6.6488e-01  4.4207e-01  1.4696e-01  3.2570e-02  5.4138e-03

Columns 7 to 12 7.1991e-04  7.9776e-05  7.5773e-06  6.2975e-07  4.6523e-08  3.0932e-09

Columns 13 to 18 1.8697e-10  1.0359e-11 -1.5363e-12 -1.0215e-12 -6.7914e-13 -4.5155e-13

Columns 19 to 20-3.0023e-13 -1.9961e-13
[ CPUFloatType{1,20} ]
<<--->>

labels.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}) = 
 5.1738
 5.0996
[ CPUFloatType{2,1} ]
<<--->>



### 定义训练函数

In [5]:
std::vector<int> x;
std::vector<double> y;
std::vector<double> y_hat;

In [6]:
torch::Tensor train(torch::Tensor train_features, 
           torch::Tensor test_features, 
           torch::Tensor train_labels, 
           torch::Tensor test_labels,
           int num_epochs = 400,
           int batch_size = 10)
{
    assertm(train_features.dim() == 2, "train_features should have 2 dims");
    assertm(test_features.dim() == 2, "test_features should have 2 dims");
    
    auto train_data_set = MyDataset(train_features, train_labels)
                                    .map(torch::data::transforms::Normalize<>(0, 0.5))
                                    .map(torch::data::transforms::Stack<>());
    auto test_data_set = MyDataset(test_features, test_labels)
                                    .map(torch::data::transforms::Normalize<>(0, 0.5))
                                    .map(torch::data::transforms::Stack<>());

    auto train_data_loader = torch::data::make_data_loader<torch::data::samplers::RandomSampler>(
                                    std::move(train_data_set), 
                                    batch_size);

    auto test_data_loader = torch::data::make_data_loader<torch::data::samplers::RandomSampler>(
                                    std::move(test_data_set), 
                                    batch_size);

    int input_shape = train_features.size(1);
    torch::nn::Sequential net({{"fc", torch::nn::Linear(torch::nn::LinearOptions(input_shape, 1).bias(false))}});
    
//     auto p = net->named_parameters(false);
//     auto w = p.find("weight");
//     auto b = p.find("bias");   
    
//     //if (w != nullptr) torch::nn::init::xavier_uniform_(*w);
//     if (w != nullptr) torch::nn::init::uniform_(*w, -1, 1);
//     //if (w != nullptr) torch::nn::init::normal_(*w);
//     if (b != nullptr) torch::nn::init::constant_(*b, 0.01);
    
//     auto optimizer = torch::optim::SGD(net->parameters(), torch::optim::SGDOptions(0.01).momentum(0.5));
    auto optimizer = torch::optim::SGD(net->parameters(), /*lr*/0.01);
    
        
    for (int epoch = 0; epoch < num_epochs; epoch++) 
    {
        torch::Tensor loss_values;
        if (epoch % 10 == 0) x.push_back(epoch);
        
        for (auto& batch : *train_data_loader) {
            auto data = batch.data;
            auto labels = batch.target;

//             optimizer.zero_grad();
            net->zero_grad();
            auto training_prediction = net->forward(data);
            loss_values = torch::mse_loss(training_prediction, labels);
            loss_values = loss_values.sum() / training_prediction.size(0);
            loss_values.backward(); 
            optimizer.step();
        }
        if (epoch % 10 == 0) 
//             y.push_back(loss_values.max().item<double>());
            y.push_back(loss_values.sum().item<double>());
        
        auto test_prediction = net->forward(test_features);
        auto test_loss_values = torch::mse_loss(test_prediction, test_labels);
        if (epoch % 10 == 0) 
//             y_hat.push_back(test_loss_values.max().item<double>());        
            y_hat.push_back(torch::sum(test_loss_values).item<double>() / test_prediction.size(0));        
        
        if (epoch % (num_epochs/10) == 0) {
        // Report the error with respect to y_training. 
        double sum_loss = loss_values.sum().item<double>();
        std::cout << "Epoch " << epoch 
            << ", sum(loss_values) = " << sum_loss << std::endl;
        }
    }
    
    std::cout << net->parameters() << std::endl;
    return net->parameters()[0];
}

In [7]:
auto train_data = 
    poly_features.index({torch::indexing::Slice(0, n_train, torch::indexing::None),
                         torch::indexing::Slice(0, 4, torch::indexing::None)});
auto train_label = 
    labels.index({torch::indexing::Slice(0, n_train, torch::indexing::None),
                         torch::indexing::Slice(0, 4, torch::indexing::None)});

auto test_data = 
    poly_features.index({torch::indexing::Slice(n_train, torch::indexing::None, torch::indexing::None),
                         torch::indexing::Slice(0, 4, torch::indexing::None)});
auto test_label = 
    labels.index({torch::indexing::Slice(n_train, torch::indexing::None, torch::indexing::None),
                         torch::indexing::Slice(0, 4, torch::indexing::None)});


printT(train_data.size(0));
printT(train_data.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}));    
printT(train_label.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}));    

train_data.size(0) = 
200
<<--->>

train_data.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}) = 
 1.0000  0.6649  0.4421  0.1470
 1.0000  0.0729  0.0053  0.0002
[ CPUFloatType{2,4} ]
<<--->>

train_label.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}) = 
 5.1738
 5.0996
[ CPUFloatType{2,1} ]
<<--->>



### 训练并验证

In [8]:
auto w = train(train_data, test_data, train_label, test_label, 5000);

Epoch 0, sum(loss_values) = 0.359569
Epoch 500, sum(loss_values) = 0.484697
Epoch 1000, sum(loss_values) = 0.0976498
Epoch 1500, sum(loss_values) = 0.381116
Epoch 2000, sum(loss_values) = 0.304709
Epoch 2500, sum(loss_values) = 0.366143
Epoch 3000, sum(loss_values) = 0.488106
Epoch 3500, sum(loss_values) = 0.475553
Epoch 4000, sum(loss_values) = 0.546498
Epoch 4500, sum(loss_values) = 0.422297
 2.0242  0.0663 -0.2096  0.3255
[ CPUFloatType{1,4} ]


In [ ]:
//true_w
// = {5.0, 1.2, -3.4, 5.6};

w = w.reshape({4,1});
printT(train_label[0]);
printT(train_data[0].reshape({1,4}).mm(w));

printT(test_label[3]);
printT(test_data[3].reshape({1,4}).mm(w));

### 训练结果可视化

In [ ]:
plt::semilogy(x, y, "b");
plt::semilogy(x, y_hat, "r");

plt::title("loss(r:test  b:train)");
plt::legend();
plt::save("./loss.png"); 
plt::show();

In [ ]:
auto img1 = im::image("./loss.png");
img1  

# 欠拟合

In [ ]:
auto train_data = 
    poly_features.index({torch::indexing::Slice(0, n_train, torch::indexing::None),
                         torch::indexing::Slice(0, 2, torch::indexing::None)});
auto train_label = 
    labels.index({torch::indexing::Slice(0, n_train, torch::indexing::None),
                         torch::indexing::Slice(0, 2, torch::indexing::None)});

auto test_data = 
    poly_features.index({torch::indexing::Slice(n_train, torch::indexing::None, torch::indexing::None),
                         torch::indexing::Slice(0, 2, torch::indexing::None)});
auto test_label = 
    labels.index({torch::indexing::Slice(n_train, torch::indexing::None, torch::indexing::None),
                         torch::indexing::Slice(0, 2, torch::indexing::None)});


printT(train_data.size(0));
printT(train_data.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}));    
printT(train_label.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}));    

In [ ]:
std::vector <int>().swap(x);
std::vector <double>().swap(y);
std::vector <double>().swap(y_hat);

auto w = train(train_data, test_data, train_label, test_label, 1000);

In [ ]:
//true_w
// = {5.0, 1.2, -3.4, 5.6};

w = w.reshape({2,1});
printT(train_label[0]);
printT(train_data[0].reshape({1,2}).mm(w));

printT(test_label[3]);
printT(test_data[3].reshape({1,2}).mm(w));

In [ ]:
plt::semilogy(x, y, "b");
plt::semilogy(x, y_hat, "r");

plt::title("loss_underfit(r:test  b:train)");
plt::legend();
plt::save("./loss_underfit.png"); 
plt::show();

In [ ]:
auto img2 = im::image("./loss_underfit.png");
img2

# 过拟合

In [ ]:
#define W_OVERFIT (20)

auto train_data = 
    poly_features.index({torch::indexing::Slice(0, n_train, torch::indexing::None),
                         torch::indexing::Slice(0, W_OVERFIT, torch::indexing::None)});
auto train_label = 
    labels.index({torch::indexing::Slice(0, n_train, torch::indexing::None),
                         torch::indexing::Slice(0, W_OVERFIT, torch::indexing::None)});

auto test_data = 
    poly_features.index({torch::indexing::Slice(n_train, torch::indexing::None, torch::indexing::None),
                         torch::indexing::Slice(0, W_OVERFIT, torch::indexing::None)});
auto test_label = 
    labels.index({torch::indexing::Slice(n_train, torch::indexing::None, torch::indexing::None),
                         torch::indexing::Slice(0, W_OVERFIT, torch::indexing::None)});


printT(train_data.size(0));
printT(train_data.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}));    
printT(train_label.index({torch::indexing::Slice(torch::indexing::None, 2, torch::indexing::None)}));    

In [ ]:
if(!x.empty()) std::vector <int>().swap(x);
if(!y.empty()) std::vector <double>().swap(y);
if(!y_hat.empty()) std::vector <double>().swap(y_hat);

auto w = train(train_data, test_data, train_label, test_label);

In [ ]:
//true_w
// = {5.0, 1.2, -3.4, 5.6};

w = w.reshape({W_OVERFIT,1});
printT(train_label[0]);
printT(train_data[0].reshape({1,W_OVERFIT}).mm(w));

printT(test_label[3]);
printT(test_data[3].reshape({1,W_OVERFIT}).mm(w));

In [ ]:
plt::semilogy(x, y, "b");
plt::semilogy(x, y_hat, "r");

plt::title("loss_overfit(r:test  b:train)");
plt::legend();
plt::save("./loss_overfit.png"); 
plt::show();

In [ ]:
auto img2 = im::image("./loss_overfit.png");
img2